In [1]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
root_dir = 'grid_data'

with open(f'{root_dir}/images.txt', 'w') as f:
    for img_file in os.listdir(f'{root_dir}/clothes'):
        f.writelines(f'{img_file.rsplit(".")[0]}\n')
f.close()
    

In [3]:
with open(f'{root_dir}/images.txt', 'r') as f:
    img_list = [line.strip() for line in f.readlines()]

print(img_list)

['00057_0', '00015_0', '00033_0', '00075_0', '00109_0', '00034_0']


## Landmarks and their codes

| Label | Code |
| --- | --- |
| Left shoulder | 10 |
| Right shoulder | 15 |
| Neck line | 13 |
| Left elbow | 25 |
| Right elbow | 32 |
| Lower chest | 29 |
| Left hand | 49 |
| Right hand | 56 |
| Middle waist | 53 |

In [4]:
def get_max_contour(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) != 0:
        max_c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(max_c)

        return max_c, (x, y, w, h)

In [5]:
def resize_thresh(img, size=(256, 256)):
    img = cv2.resize(img, size)
    return img

In [7]:
def make_grid(image, nrows=4, ncols=4):
    result = np.ones_like(image) * 255

    im_width, im_height, _ = image.shape
    row_width = int(im_width / ncols)
    row_height = int(im_height / nrows)
    
    cell_no = 0
    landmarks = {}
    for i in range(0, nrows):
        for j in range(0, ncols):
            box = (j * row_width, i * row_height, j * row_width +
                   row_width, i * row_height + row_height)
            cell_no += 1
            # if i==j or i == 0 or j == 0 or i == nrows-1 or j == ncols-1:
            if cell_no == 10 or cell_no == 13 or cell_no == 15 \
            or cell_no == 25 or cell_no == 29 or cell_no == 32 \
            or cell_no == 49 or cell_no == 53 or cell_no == 56:
                result[box[1]:box[3], box[0]:box[2]] = image[box[1]:box[3], box[0]:box[2]]
                cv2.rectangle(result, (box[0], box[1]), (box[2], box[3]), (0, 0, 0), 1)
                cv2.putText(result, f'{cell_no}', (box[0]+10, box[1]+10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
                cv2.circle(result, (box[0]+row_width//2, box[1]+row_height//2), 1, (255, 0, 0), 2)
                points = (box[0]+row_width//2, box[1]+row_height//2)
                landmarks[cell_no] = points
    
    return result, landmarks

# plt.imshow(make_grid(image=clothes_box, nrows=8, ncols=8)[0])

In [8]:
def split_image(im, rows, cols, verbose=False, output_dir=None):
    name = "grid"
    im_width, im_height = im.size
    row_width = int(im_width / cols)
    row_height = int(im_height / rows)

    n = 0
    for i in range(0, rows):
        for j in range(0, cols):
            box = (j * row_width, i * row_height, j * row_width +
                   row_width, i * row_height + row_height)
            outp = im.crop(box)
            outp_path = name + "_" + str(n) + ".png"
            outp_path = os.path.join(output_dir, outp_path)
            if verbose:
                print("Exporting image tile: " + outp_path)
            outp.save(outp_path)
            n += 1

In [9]:
clothes_landmarks = {}

for img in img_list:
    clothes = cv2.imread(f'{root_dir}/clothes/{img}.jpg')
    clothes_mask = cv2.imread(f'{root_dir}/clothes_mask/{img}.jpg', 0)
    
    contour, (x, y, w, h) = get_max_contour(mask=clothes_mask)
    
    clothes_box = clothes[y:y+h,x:x+w]
    clothes_mask_box = clothes_mask[y:y+h,x:x+w]
    
    clothes_box = resize_thresh(clothes_box)
    clothes_mask_box = resize_thresh(clothes_mask_box)
    
    clothes_box_grid, landmarks = make_grid(image=clothes_box, nrows=8, ncols=8)
    clothes_landmarks[img] = landmarks
    
    split_image(im=Image.fromarray(clothes_box), rows=8, cols=8, verbose=False, output_dir='grid_data/grids')
    
    
    # plt.figure(figsize=(15, 5))
    # plt.subplot(1, 3, 1)
    # plt.imshow(clothes_box, cmap='gray')
    # plt.subplot(1, 3, 2)
    # plt.imshow(clothes_mask_box, cmap='gray')
    # plt.subplot(1, 3, 3)
    # plt.imshow(clothes_box_grid, cmap='gray')
    # plt.show()

print(clothes_landmarks)

{'00057_0': {10: (48, 48), 13: (144, 48), 15: (208, 48), 25: (16, 112), 29: (144, 112), 32: (240, 112), 49: (16, 208), 53: (144, 208), 56: (240, 208)}, '00015_0': {10: (48, 48), 13: (144, 48), 15: (208, 48), 25: (16, 112), 29: (144, 112), 32: (240, 112), 49: (16, 208), 53: (144, 208), 56: (240, 208)}, '00033_0': {10: (48, 48), 13: (144, 48), 15: (208, 48), 25: (16, 112), 29: (144, 112), 32: (240, 112), 49: (16, 208), 53: (144, 208), 56: (240, 208)}, '00075_0': {10: (48, 48), 13: (144, 48), 15: (208, 48), 25: (16, 112), 29: (144, 112), 32: (240, 112), 49: (16, 208), 53: (144, 208), 56: (240, 208)}, '00109_0': {10: (48, 48), 13: (144, 48), 15: (208, 48), 25: (16, 112), 29: (144, 112), 32: (240, 112), 49: (16, 208), 53: (144, 208), 56: (240, 208)}, '00034_0': {10: (48, 48), 13: (144, 48), 15: (208, 48), 25: (16, 112), 29: (144, 112), 32: (240, 112), 49: (16, 208), 53: (144, 208), 56: (240, 208)}}


In [10]:
import json

# Serializing json
json_object = json.dumps(clothes_landmarks, indent=2)
 
# Writing to sample.json
with open("./grid_data/jsons/clothes_landmarks.json", "w") as outfile:
    outfile.write(json_object)
    outfile.close()

print('Write to json file successfully!')

Write to json file successfully!
